In [1]:
#ini-iPython

# Executando no diretório principal

In [2]:
import os
os.chdir('../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [3]:
# Libraries to be used.
#des-iPython import sys
#des-iPython sys.path.append('../include')
#des-iPython sys.path.append('../src')

In [4]:
# The standard libraries used in the visual memory system.


In [5]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe Basic como instanciação
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/Basic.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/Basic.py")
#end-iPython
from Basic import * # Standard and abstract class.
from Robots import * # Class responsible for performing robots tracking.
from Landmark import * # Class responsible for performing landmarks tracking.

# Class Behavior
Class responsible for managing the behavior of Visual Memory.
doxygen-iPython

In [6]:
#des-iPython class Behavior(Basic):
#ini-tab -> Iniciando código classe

---- Variables ----

# __posrobot
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [7]:
__posrobot = None #self-iPython __posrobot

# parameters
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [8]:
parameters = None #self-iPython parameters

In [10]:
#ini-iPython

# ------- Lista de funções herdadas -------



# ------- Lista variaveis -------

#self-iPython me
#self-iPython land
#self-iPython robots

#self-iPython __newrobots

# ------- Lista de funções -------

#self-iPython run
#self-iPython readDataLandmarks
#self-iPython readDataRobots
#self-iPython distributeDataRobots
#self-iPython readDataBall

#end-iPython - Fim listas

# Constructor Class
doxygen-iPython

In [11]:
#des-iPython def __init__(self):
#ini-tab -> Criando construtor
#edes-iPython super(Behavior, self).__init__("Settings", "Visual Memory")
self = Basic("Settings", "Visual Memory")

# Creating default values and reading values
parameters = {
    "number_robots": 4,
    "execution_period_ms": 100,
    "weight_robot": 10,
}
parameters = self._conf.readVariables(parameters)

# Creating objects to be tracking
me = Speeds( )
land = Landmark(me)

# Support Variables
robots = []
__newrobots = []
__posrobot = [0, 0]

for i in xrange(parameters["number_robots"]-1):
    __newrobots.append(Robots(me, __posrobot))

KALMAN FILTER - LANDMARKS section not found!


In [12]:
#end-tab -> Finalizando construtor

In [13]:
#ini-iPython

## Teste construtor

In [14]:
self._end( )
__newrobots
#end-iPython

[<Robots(Thread-4, started 140499270072064)>,
 <Robots(Thread-5, started 140499261679360)>]

# readDataLandmarks
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [15]:
#eself-iPython readDataLandmarks
#edes-iPython def readDataLandmarks(self, old):
def readDataLandmarks(old):
    if self._bkb.read_float("VISION_LAND_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_LAND_X"), self._bkb.read_float("VISION_LAND_Y")],
                "time": self._bkb.read_float("VISION_LAND_TIME"),
                "movement": self._bkb.read_float("VISION_LAND_MOV"),
            }]
        
        self._bkb.write_float("VISION_LAND_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataLandmarks

In [16]:
#ini-iPython

## Teste readDataLandmarks

In [17]:
self._bkb.write_float("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_float("VISION_LAND_TIME", 4)
self._bkb.write_float("VISION_LAND_MOV", 4)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

a: [{'tag': 1, 'pos': [2.0, 3.0], 'movement': 4.0, 'time': 4.0}]
a: [{'tag': 1, 'pos': [2.0, 3.0], 'movement': 4.0, 'time': 4.0}]


# readDataRobots
.
doxygen-iPython

In [18]:
#eself-iPython readDataRobots
#edes-iPython def readDataRobots(self, old):
def readDataRobots(old):
    data = []
    for number in xrange(1, parameters["number_robots"]):
#         print "number:", number #debug-iPython
        if self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TAG") == 0:
            continue
        
        data.append({
            "tag": self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TAG") - 2,
            "pos": [self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_X"), self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_Y")],
            "time": self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TIME"),
            "movement": self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_MOV"),
        })
        
        self._bkb.write_float("VISION_RB" + str(number).zfill(2) + "_TAG", 0)
    
    data = sorted(old + data, key= lambda k: k["time"])
    
    return data #self-iPython readDataRobots

In [19]:
#ini-iPython

## Teste readDataRobots

In [20]:
self._bkb.write_float("VISION_RB01_TAG", 2)
self._bkb.write_float("VISION_RB01_X", 1)
self._bkb.write_float("VISION_RB01_Y", 2)
self._bkb.write_float("VISION_RB01_TIME", 3)
self._bkb.write_float("VISION_RB01_MOV", 3)

a = []
a = readDataRobots(a)
print "a:", a

self._bkb.write_float("VISION_RB02_TAG", 1)
self._bkb.write_float("VISION_RB02_X", 1)
self._bkb.write_float("VISION_RB02_Y", 2)
self._bkb.write_float("VISION_RB02_TIME", 2)
self._bkb.write_float("VISION_RB02_MOV", 2)

a = readDataRobots(a)
print "a:", a
#end-iPython

a: [{'tag': 0.0, 'pos': [1.0, 2.0], 'movement': 3.0, 'time': 3.0}]
a: [{'tag': -1.0, 'pos': [1.0, 2.0], 'movement': 2.0, 'time': 2.0}, {'tag': 0.0, 'pos': [1.0, 2.0], 'movement': 3.0, 'time': 3.0}]


# distributeDataRobots
.
doxygen-iPython

In [21]:
#eself-iPython distributeDataRobots
#edes-iPython def distributeDataRobots(self, datarobots):
def distributeDataRobots(datarobots):
    if datarobots == []:
        return
    
    index = 0
    time = -1
    while index < len(datarobots):
        data = datarobots[index]
        print "Dado analisado:", data #debug-iPython
        
        # First input data
        if robots == []:
            print "Primeira leitura" #debug-iPython
            candidates = __newrobots.pop(0)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            print "\n" #debug-iPython
            continue
            
        
        if data["time"] != time:
            opponent = [ robot for robot in robots if robot.timenumber == -1 ]
            indefinite = [ robot for robot in robots if robot.timenumber == 0 ]
            teammate = [ robot for robot in robots if robot.timenumber == 1 ]
            print "Criado listas:" #debug-iPython
            print "opponent:", opponent #debug-iPython
            print "indefinite:", indefinite #debug-iPython
            print "teammate:", teammate #debug-iPython
            time = data["time"]

        
        # Calculates the similarity of the data with the objects
        #  Saving position
        __posrobot[0], __posrobot[1] = data["pos"]
        
        #  Generating Candidates
        if data["tag"] == -1:
            candidates = opponent + indefinite
        elif data["tag"] == 1:
            candidates = indefinite + teammate
        else:
            candidates = opponent + indefinite + teammate
        print "Candidatos feitos:", candidates #debug-iPython
        
        #  Calculates the similarity
        candidates.sort()
        
        #  Sends the data to the most similar object and run object update
        if (candidates == [] or candidates[0].weight > parameters["weight_robot"]) and __newrobots != []:
            print "Nenhum candidato proximo"
            candidates = __newrobots.pop(0)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            index -= 1
        else:
            if candidates[0].timenumber == -1:
                print "candidato adversario"
                opponent.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            elif candidates[0].timenumber == 1:
                print "cadidato aliado"
                teammate.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            else:
                print "candidato sei lá"
                indefinite.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
        
        print "\n" #debug-iPython
        index += 1 #self-iPython distributeDataRobots

In [22]:
#ini-iPython

## Teste distributeDataRobots

In [23]:
data = [{
    "tag": 1,
    "pos": [3, 2],
    "time": 3.0,
    "movement": 0,
},
    {
    "tag": -1,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 1,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 0,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 1,
    "pos": [4, 6],
    "time": 2.0,
    "movement": 0,
}]

data = sorted(data, key= lambda k: k["time"])

distributeDataRobots(data)

#end-iPython

Dado analisado: {'tag': -1, 'pos': [4, 6], 'movement': 0, 'time': 1.0}
Primeira leitura


Dado analisado: {'tag': 1, 'pos': [4, 6], 'movement': 0, 'time': 1.0}
Criado listas:
opponent: [<Robots(Thread-4, started 140499270072064)>]
indefinite: []
teammate: []
Candidatos feitos: []
Nenhum candidato proximo


Dado analisado: {'tag': 0, 'pos': [4, 6], 'movement': 0, 'time': 1.0}
Candidatos feitos: [<Robots(Thread-4, started 140499270072064)>]
candidato adversario


Dado analisado: {'tag': 1, 'pos': [4, 6], 'movement': 0, 'time': 2.0}
Criado listas:
opponent: [<Robots(Thread-4, started 140499270072064)>]
indefinite: []
teammate: [<Robots(Thread-5, started 140499261679360)>]
Candidatos feitos: [<Robots(Thread-5, started 140499261679360)>]
cadidato aliado


Dado analisado: {'tag': 1, 'pos': [3, 2], 'movement': 0, 'time': 3.0}
Criado listas:
opponent: [<Robots(Thread-4, started 140499270072064)>]
indefinite: []
teammate: [<Robots(Thread-5, started 140499261679360)>]
Candidatos feitos: [<Robots

# readDataBall
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [24]:
#eself-iPython readDataBall
#edes-iPython def readDataBall(self):
def readDataBall(old):
    if self._bkb.read_float("VISION_BALL_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_BALL_X"), self._bkb.read_float("VISION_BALL_Y")],
                "time": self._bkb.read_float("VISION_BALL_TIME"),
                "movement": self._bkb.read_float("VISION_BALL_MOV"),
            }]
        
        self._bkb.write_float("VISION_BALL_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataBall

In [25]:
#ini-iPython

## Teste readDataLandmarks

In [26]:
self._bkb.write_float("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_float("VISION_LAND_TIME", 4)
self._bkb.write_float("VISION_LAND_MOV", 5)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

a: [{'tag': 1, 'pos': [2.0, 3.0], 'movement': 5.0, 'time': 4.0}]
a: [{'tag': 1, 'pos': [2.0, 3.0], 'movement': 5.0, 'time': 4.0}]


# run
.
doxygen-iPython

In [27]:
#eself-iPython run
#des-iPython def run(self):
#ini-tab - Função run
# Initiating variables
datalandmarks = []
datarobots = []
databall = []

#des-iPython while True:
#ini-tab - Função while
# Start counting time
start = time.time()
print "start:", start #debug-iPython

# Reading data from landmarks
datalandmarks = readDataLandmarks(datalandmarks)

if datalandmarks != []:
    # Predict robot speed (me)
    me.update(land(datalandmarks.pop(0)))
else:
    # Predicts only the new landmarks position
    land.predict(movements = 1)

# Reading robots data
datarobots = readDataRobots(datarobots)

# Distribute the data to the robot objects
distributeDataRobots(datarobots)

# # Reading ball data
# databall = readDataBall(databall)

# # Distribute the data to the ball objects
# Batata

# # Doing cleaning and objects (lost objects)
# Batata

# # Predicts objects (Now)
# Batata

# # Send object for vision screening
# Batata

# Wait an instant on the remaining time
delta = parameters["execution_period_ms"]*1e-3 - time.time() + start
print "delta:", delta #debug-iPython
if delta > 0:
    print "Aguardando", delta*1e3, "ms" #debug-iPython
    time.sleep(delta) #self-iPython run
    print "Período finalizado" #debug-iPython   
#end-tab - Função while
#end-tab - Função run

start: 1506368134.17
delta: 0.49950003624
Aguardando 499.50003624 ms
Período finalizado


In [28]:
#ini-iPython

## Teste run

In [29]:

#end-iPython

In [30]:
#end-tab -> Finalizando classe

In [31]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [32]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [33]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [34]:
#ini-iPython

## Teste nova_funcao

In [35]:

#end-iPython

In [36]:
#end-iPython